In [10]:
import threading
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



columns = ["Website",'test_name', 'HTTP status code', "Success", "Redirected to ISP Payload"]
df = pd.DataFrame(columns=columns)

text_file = "../../../BlockingLists/russianWebsites.txt"


cyta_payload = "This site can’t be reached due tο compliance the Council Regulation (EU) 350/2022 and with EU and National Laws, only for as long as necessary."


def find_payload(text):
   if cyta_payload in text:
       return True
   else: 
       return False

In [11]:
import requests
import socket
import ssl

html_file_with_results = "result.html"
txt_file_with_results = "result.txt"
logs_file = "result.log"


def write_to_file(filename, data):
    with open(filename, "w") as f:
        f.write("%s\n" % data)




def send_request(
        host, port,
        fragment_size=0, fragment_count=0, is_https=True):
    try:
        if is_https:
            url = f"https://{host}:{port}"
        else:
            url = f"http://{host}:{port}"
        
        response = requests.get(url)
        df.loc[len(df)] = {
                "Website": url,
                "test_name": "test_warp",
                "HTTP status code": response.status_code,
                "Success": response.status_code == 200,
                "Redirected to ISP Payload": find_payload(response.text)
        }
        print(response.text)
        recv_decoded = response.text
        print(recv_decoded)
        return recv_decoded
    except requests.exceptions.ReadTimeout as e:
        print(f"ReadTimeout occurred: {e}")
        return None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None






def trucate_https(domain):
    domain = domain.replace("https://", "")
    domain = domain.replace("http://", "")
    domain = domain.replace("www.", "")
    if domain.endswith("/"):
        domain = domain[:-1]
    return domain


def get_redirection(url, timeout=3):
    try:
        response = requests.get(f"http://{url}", allow_redirects=False, timeout=timeout)
    except requests.exceptions.ConnectTimeout:
        print("Connection timed out while trying to reach the server.")
        return None, False
    max_redirects = 10
    num_redirects = 0
    redirect_url = f"http://{url}"
    is_https = False
    while response.is_redirect or response.is_permanent_redirect:
        if num_redirects >= max_redirects:
            print("Reached maximum number of redirects.")
            break

        redirect_url = response.headers['Location']
        if redirect_url.find("https") != -1:
            is_https = True
        print(f"Redirecting to: {redirect_url}")
        try:
            response = requests.get(redirect_url, allow_redirects=False, timeout=timeout)
        except requests.exceptions.ConnectTimeout:
            print("Connection timed out while trying to reach the server.")
            return None, False
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None, False
        num_redirects += 1
    redirect_url = trucate_https(redirect_url)
    print(redirect_url)
    return redirect_url, is_https


def test_http_or_https(site):
    site, is_https_required = get_redirection(site)
    if is_https_required:
        port = 443
    else:
        port = 80
    return send_request(site, port=port, is_https=is_https_required)




In [12]:
def test_sites_from_file(filename):
    with open(filename, "r") as file:
        websites = [line.strip() for line in file]

    threads = []
    for site in websites:
        thread = threading.Thread(target=process_site, args=(site,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()


def process_site(site):
    test_http_or_https(site)
    
    
# def test_sites_from_file(filename):
#     with open(filename, "r") as file:
#         websites = [line.strip() for line in file]
# 
#     for site in websites:
#         process_site(site)
# 
# 
# def process_site(site):
#     test_http_or_https(site)


test_sites_from_file(text_file)     


Redirecting to: https://iz.ru/
Redirecting to: https://svpressa.ru/
Redirecting to: https://rt.com/
Redirecting to: https://gosuslugi.ru/
Redirecting to: https://vk.com/
Redirecting to: https://regnum.ru/
Redirecting to: https://ria.ru/
Redirecting to: https://lenta.ru/
Redirecting to: https://russian.rt.com/
Redirecting to: https://vz.ru/
Redirecting to: https://www.rt.ru/
Redirecting to: https://admburla.ru/
Redirecting to: https://rg.ru


Exception in thread Thread-33 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_59239/1531773905.py", line 15, in process_site
  File "/tmp/ipykernel_59239/630748258.py", line 107, in test_http_or_https
TypeError: send_request() missing 3 required positional arguments: 'data', 'html_file', and 'txt_file'


iz.ru
Redirecting to: https://www.rt.com/
svpressa.ru
Redirecting to: https://m.vk.com/


Exception in thread Thread-32 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-36 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_59239/1531773905.py", line 15, in process_site
Exception in thread Thread-30 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstr

An error occurred: HTTPSConnectionPool(host='admburla.ru', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1007)')))
russian.rt.com
ria.ru
gosuslugi.ru


...

Exception in thread Thread-26 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_59239/1531773905.py", line 15, in process_site
  File "/tmp/ipykernel_59239/630748258.py", line 107, in test_http_or_https
TypeError: send_request() missing 3 required positional arguments: 'data', 'html_file', and 'txt_file'
Exception in thread Thread-23 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10

rt.com
m.vk.com


Exception in thread Thread-37 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-38 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-31 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/home/dmitrii/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
Exception in thread Thread-28 (process_site):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/dmitrii/.local/

Connection timed out while trying to reach the server.
Connection timed out while trying to reach the server.
Connection timed out while trying to reach the server.
Connection timed out while trying to reach the server.


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8, 8))
df['HTTP status code'].value_counts().plot(kind='pie', autopct='%1.1f%%',  colors=['blue', 'green', 'coral', 'lightskyblue', 'lightpink'])
plt.title('Distribution of Column1')
plt.ylabel('')
plt.show()



In [ ]:
plt.figure(figsize=(15, 8))

for i, column in enumerate(df.columns):
    plt.subplot(2, 3, i+1)
    if df[column].dtype == 'object':
        sns.countplot(data=df, y=column, order=df[column].value_counts().index)
    else:
        sns.histplot(data=df, x=column)
    plt.title(f'{column} Distribution')
    plt.xlabel('Count' if df[column].dtype == 'object' else 'Value')

plt.tight_layout()
plt.show()